### **Fusion Retrieval based RAG** 
1. Initial idea was to include images, but very expensive.

---

In [14]:
import os
import fitz  # PyMuPDF
import camelot
from PIL import Image
import io
from pathlib import Path

In [ ]:
def extract_text_and_tables(pdf_path):
    import os

    # If input is a folder, search for a single PDF inside it
    if os.path.isdir(pdf_path):
        input_dir = Path(pdf_path)
        pdf_files = list(input_dir.glob("*.pdf"))
        if len(pdf_files) == 0:
            raise FileNotFoundError("No PDF files found in the input directory.")
        elif len(pdf_files) > 1:
            print("Warning: Multiple PDFs found. Processing first PDF only.")
        pdf_file = pdf_files[0]
    else:
        # Direct file path
        pdf_file = Path(pdf_path)
        if not pdf_file.is_file() or pdf_file.suffix.lower() != ".pdf":
            raise FileNotFoundError("Provided file path is not a valid PDF.")

    doc = fitz.open(str(pdf_file))
    result = []

    for page_num, page in enumerate(doc, start=1):
        page_blocks = []

        blocks = page.get_text("dict")["blocks"]
        for block in blocks:
            if block["type"] == 0:
                text_content = " ".join(
                    span["text"] for line in block["lines"] for span in line["spans"]
                ).strip()
                if text_content:
                    y = block["bbox"][1]
                    page_blocks.append({
                        "type": "text",
                        "y": y,
                        "content": text_content
                    })

        try:
            tables = camelot.read_pdf(str(pdf_file), pages=str(page_num), flavor='lattice')
        except Exception as e:
            print(f"Failed to read tables on page {page_num}: {e}")
            tables = []

        for table in tables:
            table_data = table.data
            bbox = table._bbox
            y = float(bbox[1])
            page_blocks.append({
                "type": "table",
                "y": y,
                "content": table_data
            })

        page_blocks.sort(key=lambda b: b["y"])
        result.extend(page_blocks)

    return result


result = extract_text_and_tables("manual.pdf")

In [27]:
result[156]['content']

[['Description (unit of measurement)', 'T999/230V\nT999/400V'],
 ['Centres distance (mm)', '1000'],
 ['Spindle hole diameter (mm)', '38'],
 ['Maximum swing over the bed (mm)', '320'],
 ['Maximum swing over the cross slide (mm)', '198'],
 ['Turning diameter over cavity (mm)', '470'],
 ['Spindle diameter (3 + 3 self centring) (mm)', '160'],
 ['Spindle connector', 'Camlock D1-4'],
 ['No. of spindle speeds', '8\nm'],
 ['Spindle speed (r/min)', '70 - 2000 RPM'],
 ['No. of metric threads', '32'],
 ['Range of metric threads (mm)', '0.44- 10'],
 ['No. of inch threads', '20'],
 ['Range of inch threads (mm)', '2 ¼ - 40'],
 ['Range of longitudinal', '00.78- 1.044'],
 ['Range of transverse feeds (mm)', '0.022- 0.298'],
 ['Outer diameter of the feed screw (mm)', '22'],
 ['Guide length (mm)\nfervi.c o', '1390'],
 ['Cross carriage travel (mm)', '200'],
 ['Tailstock sleeve diameter (mm)', '32'],
 ['Maximum travel of the tailstock sleeve (mm)', '80'],
 ['Inner taper', 'CM 5'],
 ['Tailstock base length 